In [1]:
import torch
import torch.optim as optim
from tqdm import tqdm
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import yprov4ml
import codecarbon

# Definition of machine learning model

In [2]:
class Net(nn.Module):
    def __init__(self, model_size, dropout):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

        def get_layer_sizes(model_size): 
            if model_size == "small": 
                return 64, 32
            elif model_size == "medium": 
                return 512, 256
            else: 
                return 1024, 256

        l1, l2 = get_layer_sizes(model_size)

        self.fc1 = nn.Linear(12544, l1)
        self.fc2 = nn.Linear(l1, l2)
        self.fc3 = nn.Linear(l2, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.dropout1(x)
        x = self.fc3(x)
        output = F.log_softmax(x, dim=1)
        return output

In [3]:
def train(lr, epochs, batch_size, dropout, model_size):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    trainloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)

    model = Net(model_size, dropout=dropout).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = torch.nn.CrossEntropyLoss().to(device)
    scheduler = None

    model.train()

    losses = []
    for _ in range(epochs): 
        for data in tqdm(trainloader):
            inputs, labels = data[0].to(device), data[1].to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            losses.append(loss.item())
            loss.backward()
            optimizer.step()
            if scheduler is not None:
                scheduler.step()

    return model

def validate(model, batch_size=128):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
    testloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=False)
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in tqdm(testloader):
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy: {100 * correct / total} %')
    return correct / total

We define the objective function that will be optimized by the hyperparameter optimization library.
Here we also want to track the carbon emissions during the training process.

In [4]:
def objective_function(params):
    lr = params["lr"]
    epochs = params["epochs"]
    batch_size = params["batch_size"]
    dropout = 0.25
    model_size = "small"

    tracker = codecarbon.EmissionsTracker(
        save_to_api=False,
        save_to_file=False,
        save_to_logger=False,
        log_level="error",
    )
    tracker.start_task()
    model = train(lr, epochs, batch_size, dropout, model_size)
    emissions = tracker.stop_task()
    
    accuracy = validate(model, batch_size)

    return {"accuracy": accuracy, "energy_consumed": emissions.energy_consumed}

In [5]:
# # This cell is used to call the objective function (train the model) to generate some provenance data
# res = objective_function({
#     "lr": 0.001,
#     "epochs": 5,
#     "batch_size": 32
# })

# yprov4ml.start_run(
#     prov_user_namespace="www.example.org",
#     experiment_name=f"0.001_5_32_experiment", 
#     provenance_save_dir="workflow_prov",
#     save_after_n_logs=100,
#     collect_all_processes=False, 
#     disable_codecarbon=True, 
#     metrics_file_type=yprov4ml.MetricsType.NETCDF,
# )

    
# yprov4ml.log_param("lr", 0.001, yprov4ml.Context.TRAINING)
# yprov4ml.log_param("epochs", 5, yprov4ml.Context.TRAINING)
# yprov4ml.log_param("batch_size", 32, yprov4ml.Context.TRAINING)

# yprov4ml.log_metric("accuracy", res["accuracy"], yprov4ml.Context.VALIDATION)
# yprov4ml.log_metric("energy_consumed", res["energy_consumed"], yprov4ml.Context.TRAINING)

# yprov4ml.end_run(
#     create_graph=False,
#     create_svg=False,
#     crate_ro_crate=False
# )

In [6]:
from library import Experiment, SearchSpace, FloatParameter, IntParameter, OptimizationParameters, OptimizerConfig

/Users/cristianmurtas/miniconda3/envs/newtest/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


We first define which parameters we want to optimize and their bounds. This will be used to create the search space for the optimization.
The output list will contain the metrics we want to optimize, in this case accuracy and energy consumed.
The directions list will contain the direction of optimization for each output metric, in this case we want to maximize accuracy and minimize energy consumed.

In [7]:
lr = FloatParameter("lr", 0.0001, 0.001)
batch_size = IntParameter("batch_size", 16, 32)
epochs = IntParameter("epochs", 5, 10)
search_space = SearchSpace(parameters=[lr, batch_size, epochs])

opt_params = OptimizationParameters(
    input=search_space,
    output=["accuracy", "energy_consumed"],
    directions=["max", "min"]
)

optimizer_config = OptimizerConfig(
    200, 10, "ucb", 1.0
)

Then we set up the Experiment (which represents an optimization task) and run the optimize function with a given objective function

In [8]:
exp = Experiment(
    optimization_parameters=opt_params,
    optimizer_config=optimizer_config,
    path_to_prov="./workflow_prov",
    n_iter=1
)

exp.optimize(objective_function)

{'lr': {'$': 0.001, 'type': 'xsd:double'}, 'batch_size': {'$': 32, 'type': 'xsd:int'}, 'epochs': {'$': 5, 'type': 'xsd:int'}, 'accuracy': {'yProv4ML:label': 'accuracy', 'yProv4ML:path': 'workflow_prov/0.001_5_32_experiment_0/metrics_GR0/accuracy_Context.VALIDATION_GR0.nc', 'yProv4ML:role': 'output', 'yProv4ML:context': 'Context.VALIDATION', 'yProv4ML:source': 'None'}, 'energy_consumed': {'yProv4ML:label': 'energy_consumed', 'yProv4ML:path': 'workflow_prov/0.001_5_32_experiment_0/metrics_GR0/energy_consumed_Context.TRAINING_GR0.nc', 'yProv4ML:role': 'output', 'yProv4ML:context': 'Context.TRAINING', 'yProv4ML:source': 'None'}}
{'lr': {'$': 0.0006776156295803325, 'type': 'xsd:double'}, 'batch_size': {'$': 32, 'type': 'xsd:int'}, 'epochs': {'$': 5, 'type': 'xsd:int'}, 'accuracy': {'yProv4ML:label': 'accuracy', 'yProv4ML:path': './workflow_prov/test_prov_experiment_1/metrics_GR0/accuracy_Context.TRAINING_GR0.nc', 'yProv4ML:role': 'output', 'yProv4ML:context': 'Context.TRAINING', 'yProv4ML:s

[codecarbon WARNING @ 09:23:44] Multiple instances of codecarbon are allowed to run at the same time.
100%|██████████| 323/323 [00:03<00:00, 87.92it/s]


Accuracy: 67.16 %
Evaluated candidate: {'lr': 0.001018, 'batch_size': 31, 'epochs': 6} Result: {'accuracy': 0.6716, 'energy_consumed': 0.0003227162497267158}
Logging config to provenance: {'lr': 0.001018, 'batch_size': 31, 'epochs': 6}
Logging results to provenance: {'accuracy': 0.6716, 'energy_consumed': 0.0003227162497267158}


In [10]:
print(exp.results())

{'lr': {'$': 0.001, 'type': 'xsd:double'}, 'batch_size': {'$': 32, 'type': 'xsd:int'}, 'epochs': {'$': 5, 'type': 'xsd:int'}, 'accuracy': {'yProv4ML:label': 'accuracy', 'yProv4ML:path': 'workflow_prov/0.001_5_32_experiment_0/metrics_GR0/accuracy_Context.VALIDATION_GR0.nc', 'yProv4ML:role': 'output', 'yProv4ML:context': 'Context.VALIDATION', 'yProv4ML:source': 'None'}, 'energy_consumed': {'yProv4ML:label': 'energy_consumed', 'yProv4ML:path': 'workflow_prov/0.001_5_32_experiment_0/metrics_GR0/energy_consumed_Context.TRAINING_GR0.nc', 'yProv4ML:role': 'output', 'yProv4ML:context': 'Context.TRAINING', 'yProv4ML:source': 'None'}}
{'lr': {'$': 0.001018, 'type': 'xsd:double'}, 'batch_size': {'$': 31, 'type': 'xsd:int'}, 'epochs': {'$': 6, 'type': 'xsd:int'}, 'accuracy': {'yProv4ML:label': 'accuracy', 'yProv4ML:path': './workflow_prov/test_prov_experiment_2/metrics_GR0/accuracy_Context.TRAINING_GR0.nc', 'yProv4ML:role': 'output', 'yProv4ML:context': 'Context.TRAINING', 'yProv4ML:source': 'None